In [1]:
import sklearn as skl
import matplotlib.pyplot as plt
from sklearn import svm, preprocessing, metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import scipy

import numpy as np, random, json, pickle, datetime, copy, socket, os, sys
from scipy.stats import sem
import matplotlib.colors as colors
from importlib import reload

if socket.gethostname() == 'Tolman':
    codeDirBase = 'C:\\Users\\whockei1\\Google Drive'
elif socket.gethostname() == 'DESKTOP-BECTOJ9':
    codeDirBase = 'C:\\Users\\whock\\Google Drive'
    
sys.path.insert(0, codeDirBase + '\\KnierimLab\\Ratterdam\\Code')
import utility_fx as util
import ratterdam_ParseBehavior as Parse
import ratterdam_CoreDataStructures as Core
import ratterdam_PermutationTests as Perm
import ratterdam_Defaults as Def

from itertools import cycle

from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from scipy import interp
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels

In [2]:
%qtconsole --style native

In [2]:
# figpath = f"C:\\Users\\whockei1\\Google Drive\\KnierimLab\\Ratterdam\\Figures\\R765{exp}\\"
# datafile = f'E:\\Ratterdam\\R765\\R765{exp}\\'
# behav = core.BehavioralData(datafile, exp, velocity_filter_thresh)
# ts, position, alleyTracking, alleyVisits,  txtVisits = behav.loadData()

datafile = "E:\\Ratterdam\\R781\\Beltway_D3_190307\\"
expCode = "BRD3"
alleyTracking, alleyVisits,  txtVisits, p_sess, ts_sess = Parse.getDaysBehavioralData(datafile, expCode)
population = {}
for subdir, dirs, fs in os.walk(datafile):
    for f in fs:
        if 'cl-maze1' in f and 'OLD' not in f and 'Undefined' not in f:
            clustname = subdir[subdir.index("TT"):] + "\\" + f
            print(clustname)
            unit = Core.UnitData(clustname, datafile, expCode, Def.alleyBounds, alleyVisits, txtVisits, p_sess, ts_sess)
            unit.loadData_raw(includeRewards=False)
            population[unit.name] = unit

TT11\cl-maze1.1


C:\Users\whockei1\Google Drive\KnierimLab\Ratterdam\Code\ratterdam_CoreDataStructures.py:157: RuntimeWarning: divide by zero encountered in reciprocal
  n = (hs*np.reciprocal(ho))*30
C:\Users\whockei1\Google Drive\KnierimLab\Ratterdam\Code\ratterdam_CoreDataStructures.py:157: RuntimeWarning: invalid value encountered in multiply
  n = (hs*np.reciprocal(ho))*30
C:\Users\whockei1\Google Drive\KnierimLab\Ratterdam\Code\ratterdam_CoreDataStructures.py:163: RuntimeWarning: divide by zero encountered in reciprocal
  n = (ls* np.reciprocal(lo)) * 30
C:\Users\whockei1\Google Drive\KnierimLab\Ratterdam\Code\ratterdam_CoreDataStructures.py:163: RuntimeWarning: invalid value encountered in multiply
  n = (ls* np.reciprocal(lo)) * 30
C:\Users\whockei1\Google Drive\KnierimLab\Ratterdam\Code\utility_fx.py:321: RuntimeWarning: invalid value encountered in true_divide
  Z=VV/WW


TT11\cl-maze1.2
TT3\cl-maze1.1
TT3\cl-maze1.2
TT3\cl-maze1.3
TT3\cl-maze1.4
TT3\cl-maze1.5
TT3\cl-maze1.6
TT3\cl-maze1.7
TT5\cl-maze1.1
TT5\cl-maze1.2
TT5\cl-maze1.3
TT5\cl-maze1.4
TT6\cl-maze1.1
TT6\cl-maze1.2
TT6\cl-maze1.3
TT6\cl-maze1.4
TT6\cl-maze1.5
TT6\cl-maze1.6
TT6\cl-maze1.7
TT6\cl-maze1.8
TT6\cl-maze1.9
TT9\cl-maze1.1
TT9\cl-maze1.2
TT9\cl-maze1.3
TT9\cl-maze1.4
TT9\cl-maze1.5
TT9\cl-maze1.6
TT9\cl-maze1.7


#### Due to temporal confound, recompute average texture-response linear ratemap ONLY WITH trials meeting a firing criteria of >1hz in >3 bins (10% have some activity). 
#### The rationale is then a cell will not "vote" that it does not "like" a texture simply because it did not have a field when that texture was present.

In [119]:
firing_thresh = 1# what is minimum firing rate in Hz required
spike_thresh = 5
bin_num_thresh = 3 # in how many bins, at minimum, is the above rate required? Not nec. contiguous. 

for unitName in unitList:
    unit = population[unitName]
    
    print(unitName)

    for alley in range(1,18):
        
        for texture in ['A', 'B', 'C']:
            
            visit_spikes = np.empty((0,3))
            visit_occs = np.empty((0,3))
            added_ratemap = False # toggle if any rms have passed and added their spikes. IDK how many spikes that is so i cant just check shape
            c = 0
            for visit in unit.alleys[alley]:
                if visit['metadata']['stimulus'][0] == texture:
                    ratemap = visit['ratemap1d']
                    if ratemap[ratemap>firing_thresh].shape[0] >= bin_num_thresh:   #key line - this is the exclusion
                    #if visit['spikes'].shape[0] > spike_thresh:
                        visit_spikes = np.vstack((visit_spikes, visit['spikes']))
                        visit_occs = np.vstack((visit_occs, visit['occs']))
                        added_ratemap = True
                        c+=1

            if added_ratemap:
                unit.linRMS[alley][texture] = unit.computeSingleRM(visit_spikes, visit_occs, alley, dim=1)
            else:
                unit.linRMS[alley][texture] = None
                
        # if any are none, delete the whole alley's data 
        if unit.linRMS[alley]['A'] is None or unit.linRMS[alley]['B'] is None or unit.linRMS[alley]['C'] is None:
            for txt in ['A','B','C']:
                unit.linRMS[alley][txt] = None
            
        

TT10cl-maze1.1
TT10cl-maze1.2
TT10cl-maze1.3
TT10cl-maze1.4
TT10cl-maze1.5
TT10cl-maze1.6
TT10cl-maze1.7
TT13cl-maze1.1
TT13cl-maze1.2
TT13cl-maze1.3
TT15cl-maze1.1
TT15cl-maze1.2
TT15cl-maze1.3
TT15cl-maze1.4
TT15cl-maze1.5
TT4cl-maze1.1


C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in greater
C:\Users\whockei1\Google Drive\KnierimLab\Ratterdam\Code\ratterdam_CoreDataStructures.py:145: RuntimeWarning: divide by zero encountered in reciprocal
  n = (ls* np.reciprocal(lo)) * 33
C:\Users\whockei1\Google Drive\KnierimLab\Ratterdam\Code\ratterdam_CoreDataStructures.py:145: RuntimeWarning: invalid value encountered in multiply
  n = (ls* np.reciprocal(lo)) * 33
C:\Users\whockei1\Google Drive\Python_Code\KLab\mts_analysis\utility_fx.py:309: RuntimeWarning: invalid value encountered in true_divide
  Z=VV/WW
C:\Users\whockei1\Google Drive\Python_Code\KLab\mts_analysis\utility_fx.py:305: RuntimeWarning: invalid value encountered in multiply
  W=0*U.copy()+1


TT4cl-maze1.10
TT4cl-maze1.11
TT4cl-maze1.12
TT4cl-maze1.2
TT4cl-maze1.3
TT4cl-maze1.4
TT4cl-maze1.5
TT4cl-maze1.6
TT4cl-maze1.7
TT4cl-maze1.8
TT4cl-maze1.9
TT6cl-maze1.1
TT6cl-maze1.2
TT7cl-maze1.1
TT7cl-maze1.2
TT7cl-maze1.3


## Texture x Alley Conjunctive Decoding (using separate random forests at each alley)

In [76]:
all_oobs, all_cvs = [], []

ratemaps = {txt: np.empty((0, Def.singleAlleyBins[0]-1)) for txt in ['A', 'B', 'C']}

for alley in [1,3,5,7,8,10,11,16,17]:
    for clustName, unit in population.items():
        for stim in ['A', 'B', 'C']:
            rm = unit.linRMS[alley][stim]
            if type(rm) == np.ndarray:
                ratemaps[stim] = np.vstack((ratemaps[stim], rm))
for stim in ['A','B','C']:
    ratemaps[stim][np.where(np.isnan(ratemaps[stim]))] = 0

pop = np.vstack((ratemaps['A'], ratemaps['B'], ratemaps['C']))
labels = ['A']*ratemaps['A'].shape[0] + ['B']*ratemaps['B'].shape[0] + ['C']*ratemaps['C'].shape[0]
Y = np.asarray(labels)
for r in range(15):
    print(f"iter {r}")
    oobs, cvs = [], []
    
    for alley in [1,3,5,7,8,10,11,16,17]:
        print(f"alley {alley}")
        
        ratemaps = {txt: np.empty((0, Def.singleAlleyBins[0]-1)) for txt in ['A', 'B', 'C']}
        for clustName, unit in population.items():
            for stim in ['A', 'B', 'C']:
                rm = unit.linRMS[alley][stim]
                if type(rm) == np.ndarray:
                    ratemaps[stim] = np.vstack((ratemaps[stim], rm))
        for stim in ['A','B','C']:
            ratemaps[stim][np.where(np.isnan(ratemaps[stim]))] = 0
            
        pop = np.vstack((ratemaps['A'], ratemaps['B'], ratemaps['C']))
        labels = ['A']*ratemaps['A'].shape[0] + ['B']*ratemaps['B'].shape[0] + ['C']*ratemaps['C'].shape[0]
        if pop.shape[0] > 0:
            X = preprocessing.StandardScaler().fit_transform(pop)
            Y = np.asarray(labels)

            clf = RandomForestClassifier(n_estimators=700, oob_score=True)
            clf2 = RandomForestClassifier(n_estimators=700, oob_score=False)
            Xtrain, Xtest, ytrain, ytest = train_test_split(X,Y,random_state=0)
            clf2.fit(Xtrain,ytrain)
            ypred = clf2.predict(Xtest)
            cv = np.where(ypred==ytest)[0].shape[0]/len(ypred)
            clf.fit(X,Y)
            
            cvs.append(np.mean(cv))
            clf.fit(X,Y)
            oobs.append(clf.oob_score_)
    all_oobs.append(np.mean(oobs))
    all_cvs.append(np.mean(cvs))
        

iter 0
alley 1
alley 3
alley 5
alley 7
alley 8
alley 10
alley 11
alley 16
alley 17
iter 1
alley 1
alley 3
alley 5
alley 7
alley 8
alley 10
alley 11
alley 16
alley 17
iter 2
alley 1
alley 3
alley 5
alley 7
alley 8
alley 10
alley 11
alley 16
alley 17
iter 3
alley 1
alley 3
alley 5
alley 7
alley 8
alley 10
alley 11
alley 16
alley 17
iter 4
alley 1
alley 3
alley 5
alley 7
alley 8
alley 10
alley 11
alley 16
alley 17
iter 5
alley 1
alley 3
alley 5
alley 7
alley 8
alley 10
alley 11
alley 16
alley 17
iter 6
alley 1
alley 3
alley 5
alley 7
alley 8
alley 10
alley 11
alley 16
alley 17
iter 7
alley 1
alley 3
alley 5
alley 7
alley 8
alley 10
alley 11
alley 16
alley 17
iter 8
alley 1
alley 3
alley 5
alley 7
alley 8
alley 10
alley 11
alley 16
alley 17
iter 9
alley 1
alley 3
alley 5
alley 7
alley 8
alley 10
alley 11
alley 16
alley 17
iter 10
alley 1
alley 3
alley 5
alley 7
alley 8
alley 10
alley 11
alley 16
alley 17
iter 11
alley 1
alley 3
alley 5
alley 7
alley 8
alley 10
alley 11
alley 16
alley 17
it

In [77]:
plt.bar(1, np.mean(all_cvs), width=0.2, color='cornflowerblue', yerr=np.std(all_cvs), alpha=0.6, align='edge')
plt.scatter([1]*len(all_cvs), all_cvs, c='k', alpha=0.3)
plt.bar(1.5, np.mean(all_oobs), width=0.2, color='firebrick', yerr=np.std(all_oobs), alpha=0.6)
plt.scatter([1.5]*len(all_oobs), all_oobs, c='k', alpha=0.3)
plt.xticks([1, 1.5], ["Cross validation", "OOB"], fontsize=22)
plt.ylabel("Performance", fontsize=32)
plt.yticks(fontsize=22)

(array([ 0. ,  0.1,  0.2,  0.3,  0.4,  0.5]),
 <a list of 6 Text yticklabel objects>)

In [80]:
b1, b2 = 1,1.5

titles = ["Alley Alone", "Texture Alone", "Texture within Alley", "All Alley / Texture Combos"]
fig, ax = plt.subplots(2,2)
for i,c,o,n,t in zip([0,1,2,3],[cvsAlley, cvsTxt, all_cvs, cvsCombo], [oobsAlley, oobsTxt, all_oobs, oobsCombo],[1/9, 1/3, 1/3, 1/27], titles):
    fig.axes[i].bar(b1, np.mean(c), width=0.2, color='cornflowerblue', yerr=np.std(c), alpha=0.6, align='edge')
    fig.axes[i].bar(b2, np.mean(o), width=0.2, color='firebrick', yerr=np.std(o), alpha=0.6, align='edge')
    
    fig.axes[i].scatter([b1]*len(c), c, c='k', alpha=0.3)
    fig.axes[i].scatter([b2]*len(o), o, c='k', alpha=0.3)
    
    fig.axes[i].set_xticks([b1,b2])
    fig.axes[i].set_xticklabels(["Cross-Validation", "OOB"], fontsize=22)
    fig.axes[i].tick_params(labelsize=22)
    fig.axes[i].set_ylabel("Performance", fontsize=32)
    
    fig.axes[i].set_title(t, fontsize=22)
    
    xlims = fig.axes[i].get_xlim()
    fig.axes[i].plot([xlims[0], xlims[1]], [n, n], 'k--')

In [22]:
%qtconsole --style paraiso-dark

## Texture Alone Decoding

In [74]:
# txt alone
ratemaps = {txt: np.empty((0, Def.singleAlleyBins[0]-1)) for txt in ['A', 'B', 'C']}

for alley in [1,3,5,7,8,10,11,16,17]:
    for clustName, unit in population.items():
        for stim in ['A', 'B', 'C']:
            rm = unit.linRMS[alley][stim]
            if type(rm) == np.ndarray:
                ratemaps[stim] = np.vstack((ratemaps[stim], rm))
    for stim in ['A','B','C']:
        ratemaps[stim][np.where(np.isnan(ratemaps[stim]))] = 0

pop = np.vstack((ratemaps['A'], ratemaps['B'], ratemaps['C']))
labels = ['A']*ratemaps['A'].shape[0] + ['B']*ratemaps['B'].shape[0] + ['C']*ratemaps['C'].shape[0]
X = preprocessing.StandardScaler().fit_transform(pop)
Y = np.asarray(labels)
    
oobsTxt,cvsTxt = [], []   

for i in range(15):
    clf = RandomForestClassifier(n_estimators=700, oob_score=True)
    clf2 = RandomForestClassifier(n_estimators=700, oob_score=False)
    Xtrain, Xtest, ytrain, ytest = train_test_split(X,Y,random_state=0)
    clf2.fit(Xtrain,ytrain)
    ypred = clf2.predict(Xtest)
    cvs = np.where(ypred==ytest)[0].shape[0]/len(ypred)
    clf.fit(X,Y)
    oobsTxt.append(clf.oob_score_)
    cvsTxt.append(np.mean(cvs))


In [75]:
plt.bar(1, np.mean(cvsTxt), color='cornflowerblue', yerr=np.std(cvsTxt), alpha=0.6)
plt.scatter([1]*len(cvsTxt), cvsTxt, c='k', alpha=0.3)
plt.bar(2, np.mean(oobsTxt), color='firebrick', yerr=np.std(oobsTxt), alpha=0.6)
plt.scatter([2]*len(oobsTxt), oobsTxt, c='k', alpha=0.3)
plt.xticks([1,2,3], ["Accuracy", "OOB"], fontsize=22)
plt.ylabel("Score", fontsize=32)
plt.title("Texture Alone Classification", fontsize=28)
plt.show()

## Alley Alone Decoding

In [71]:
# alley alone
ratemaps = {alley: np.empty((0, Def.singleAlleyBins[0]-1)) for alley in [1,3,5,7,8,10,11,16,17]}

for alley in [1,3,5,7,8,10,11,16,17]:
    for clustName, unit in population.items():
        for stim in ['A', 'B', 'C']:
            rm = unit.linRMS[alley][stim]
            if type(rm) == np.ndarray:
                ratemaps[alley] = np.vstack((ratemaps[alley], rm))
    for alley in [1,3,5,7,8,10,11,16,17]:
        ratemaps[alley][np.where(np.isnan(ratemaps[alley]))] = 0

pop = np.vstack(([ratemaps[i] for i in [1,3,5,7,8,10,11,16,17]]))
labels = [[i]*ratemaps[i].shape[0] for i in [1,3,5,7,8,10,11,16,17]]
labels = np.asarray([item for sublist in labels for item in sublist])
X = preprocessing.StandardScaler().fit_transform(pop)
Y = np.asarray(labels)
cvsAlley, oobsAlley = [], []  
#cvs = cross_val_score(clf, X, Y, cv=5)
for i in range(10):
    clf = RandomForestClassifier(n_estimators=700, oob_score=True)
    clf2 = RandomForestClassifier(n_estimators=700, oob_score=False)
    Xtrain, Xtest, ytrain, ytest = train_test_split(X,Y,random_state=0)
    clf2.fit(Xtrain,ytrain)
    ypred = clf2.predict(Xtest)
    cvs = np.where(ypred==ytest)[0].shape[0]/len(ypred)
    clf.fit(X,Y)
    cvsAlley.append(np.mean(cvs))
    oobsAlley.append(clf.oob_score_)


In [72]:
plt.bar(1, np.mean(cvsAlley), color='cornflowerblue', yerr=np.std(cvsAlley), alpha=0.6)
plt.scatter([1]*len(cvsAlley), cvsAlley, c='k', alpha=0.3)
plt.bar(2, np.mean(oobsAlley), color='firebrick', yerr=np.std(oobsAlley), alpha=0.6)
plt.scatter([2]*len(oobsAlley), oobsAlley, c='k', alpha=0.3)
plt.xticks([1,2], ["Cross Validation", "OOB"], fontsize=22)
plt.ylabel("Score", fontsize=32)
plt.title("Alley Alone Classification", fontsize=24)
plt.yticks(fontsize=26)
plt.show()

## All condition decoding (all texture x alley combinations - 51 label decoding)

In [11]:
ratemaps = {alley: np.empty((0, Def.singleAlleyBins[0]-1)) for alley in [16,17,3,1,5,7,8,10,11]}
labels = []
popidx = []
idx = 0
# accs, shuffs, oobs = [], [], []
cvsCombo, oobsCombo = [], []
for alley in [16,17,3,1,5,7,8,10,11]:
    for clustName, unit in population.items():
        popidx.append(idx)
        for stim in ['A', 'B', 'C']:
            rm = unit.linRMS[alley][stim]
            if type(rm) == np.ndarray:
                ratemaps[alley] = np.vstack((ratemaps[alley], rm))
                labels.append(f"{alley}{stim}")
                idx += 1
    for alley in [16,17,3,1,5,7,8,10,11]:
        ratemaps[alley][np.where(np.isnan(ratemaps[alley]))] = 0
        
pop = np.vstack(([ratemaps[i] for i in [16,17,3,1,5,7,8,10,11]]))
X = preprocessing.StandardScaler().fit_transform(pop)
Y = np.asarray(labels)
accsCombo = []
oobsCombo = []
shuffsCombo = []
clf = RandomForestClassifier(n_estimators=700, oob_score=True)
for i in range(10):
    clf = RandomForestClassifier(n_estimators=700, oob_score=True)
    clf2 = RandomForestClassifier(n_estimators=1000, oob_score=False)

    #cvs = cross_val_score(clf, X, Y, cv=kf)
    Xtrain, Xtest, ytrain, ytest = train_test_split(X,Y,random_state=0)
    clf2.fit(Xtrain,ytrain)
    ypred = clf2.predict(Xtest)
    cvs = np.where(ypred==ytest)[0].shape[0]/len(ypred)
    clf.fit(X,Y)
    cvsCombo.append(np.mean(cvs))
    oobsCombo.append(clf.oob_score_)

In [104]:
plt.bar(1, np.mean(cvsCombo), color='cornflowerblue', yerr=np.std(cvsCombo), alpha=0.6)
plt.scatter([1]*len(cvsCombo), cvsCombo, c='k', alpha=0.3)
plt.bar(2, np.mean(oobsCombo), color='firebrick', yerr=np.std(oobsCombo), alpha=0.6)
plt.scatter([2]*len(oobsCombo), oobsCombo, c='k', alpha=0.3)
plt.xticks([1,2,3], ["Accuracy", "OOB"], fontsize=22)
plt.ylabel("Score", fontsize=32)
plt.title("Classification of All Alley/Texure Combinations", fontsize=24)
plt.show()

## Dropout Analysis
### Want to know how performance of stimulus decoding and position decoding are affected by dropping out different Units.

In [47]:
# txt alone

# alley_scores = []
# txt_scores = []
for clust in ['xxx']:
    print(f"Dropping out {clust}")
    
    ratemaps = {txt: np.empty((0, 30)) for txt in ['A', 'B', 'C']}
    
    print("Attempting texture decoding")
    
    for alley in range(1,18):
        for clustName, unit in population.items():
            if clustName != clust:
                for stim in ['A', 'B', 'C']:
                    rm = unit.linRMS[alley][stim]
                    if type(rm) == np.ndarray:
                        ratemaps[stim] = np.vstack((ratemaps[stim], rm))
        for stim in ['A','B','C']:
            ratemaps[stim][np.where(np.isnan(ratemaps[stim]))] = 0

        pop = np.vstack((ratemaps['A'], ratemaps['B'], ratemaps['C']))
        labels = ['A']*ratemaps['A'].shape[0] + ['B']*ratemaps['B'].shape[0] + ['C']*ratemaps['C'].shape[0]
        X = preprocessing.StandardScaler().fit_transform(pop)
        Y = np.asarray(labels)

    clf = RandomForestClassifier(n_estimators=700, oob_score=True)
    cvs_alley = cross_val_score(clf, X, Y, cv=10)
    #txt_scores.append(cvs)


    # alley alone
    ratemaps = {alley: np.empty((0, 30)) for alley in range(1,18)}
    
    print("Attempting alley decoding")

    for alley in range(1,18):
        for clustName, unit in population.items():
            if clustName != clust:
                for stim in ['A', 'B', 'C']:
                    rm = unit.linRMS[alley][stim]
                    if type(rm) == np.ndarray:
                        ratemaps[alley] = np.vstack((ratemaps[alley], rm))
        for alley in range(1,18):
            ratemaps[alley][np.where(np.isnan(ratemaps[alley]))] = 0

        pop = np.vstack(([ratemaps[i] for i in range(1,18)]))
        labels = [[i]*ratemaps[i].shape[0] for i in range(1,18)]
        labels = np.asarray([item for sublist in labels for item in sublist])
        X = preprocessing.StandardScaler().fit_transform(pop)
        Y = np.asarray(labels)

    clf = RandomForestClassifier(n_estimators=200, oob_score=True)
    clf.fit(X,Y)
    cvs_txt = cross_val_score(clf, X, Y, cv=10)
    #alley_scores.append(cvs)

Dropping out xxx
Attempting texture decoding
Attempting alley decoding


In [41]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [37]:
alley_means  = np.asarray([np.mean(i) for i in alley_scores])
alley_errs = np.asarray([sem(i) for i in alley_scores])

txt_means  = np.asarray([np.mean(i) for i in txt_scores])
txt_errs = np.asarray([sem(i) for i in txt_scores])


In [51]:
plt.plot(alley_means, "g-^")
#plt.fill_between(range(len(alley_means)), alley_means-alley_errs, alley_means+alley_errs, color="g", alpha=0.5)

plt.plot(txt_means, "b-^")
#plt.fill_between(range(len(txt_means)), txt_means-txt_errs, txt_means+txt_errs, color="b", alpha=0.5)

plt.plot([np.mean(alley_means)]*len(alley_means),'g')
plt.plot([np.mean(txt_means)]*len(txt_means),'b')

## Txt and Alley Pop Matrices for Exploration

In [90]:
ratemaps_txt = {txt: np.empty((0, 30)) for txt in ['A', 'B', 'C']}
for alley in range(1,18):
    for clustName, unit in population.items():
        if clustName != clust:
            for stim in ['A', 'B', 'C']:
                rm = unit.linRMS[alley][stim]
                if type(rm) == np.ndarray:
                    ratemaps_txt[stim] = np.vstack((ratemaps_txt[stim], rm))
    for stim in ['A','B','C']:
        ratemaps_txt[stim][np.where(np.isnan(ratemaps_txt[stim]))] = 0

    pop_txts = np.vstack((ratemaps_txt['A'], ratemaps_txt['B'], ratemaps_txt['C']))
    labels_txts = ['A']*ratemaps_txt['A'].shape[0] + ['B']*ratemaps_txt['B'].shape[0] + ['C']*ratemaps_txt['C'].shape[0]


    
ratemaps_alleys = {alley: np.empty((0, 30)) for alley in range(1,18)}
for alley in range(1,18):
    for clustName, unit in population.items():
        if clustName != clust:
            for stim in ['A', 'B', 'C']:
                rm = unit.linRMS[alley][stim]
                if type(rm) == np.ndarray:
                    ratemaps_alleys[alley] = np.vstack((ratemaps_alleys[alley], rm))
    for alley in range(1,18):
        ratemaps_alleys[alley][np.where(np.isnan(ratemaps_alleys[alley]))] = 0

    pop_alleys = np.vstack(([ratemaps_alleys[i] for i in range(1,18)]))
    labels = [[i]*ratemaps_alleys[i].shape[0] for i in range(1,18)]
    labels_alleys = np.asarray([item for sublist in labels for item in sublist])

In [104]:
frs = []
for entry in pop_alleys:
    entry = np.ndarray.flatten(entry)
    frs.extend(entry)
frs = np.asarray(frs)
frs = frs[np.isfinite(frs)]
h,b = np.histogram(frs, bins=100)
frcum = np.cumsum(h)
propExp = np.asarray([i/h.sum() for i in frcum])
try:
    thresh = np.where(propExp < cutoff)[0][-1]
except:
    thresh = np.where(b == np.median(b))
_max = b[thresh]*1.

In [114]:

fig, ax = plt.subplots(1,3)
for i,txt in zip([0,1,2],['A','B','C']):
    sortedArray = np.sort(ratemaps_txt[txt],axis=0)
    ax[i].imshow(np.sort(sortedArray), aspect='auto', interpolation='None',vmax=_max, origin='lower')
    ax[i].set_title(txt)
    ax[i].set_ylim([sortedArray.shape[0], sortedArray.shape[0]-25])

# Distributed Texture Info: Binned test stats across pop

In [2]:
exp = "RFD5"
population = {}
dayCode = f"R765{exp}\\"
figpath = f"C:\\Users\\whockei1\\Google Drive\\KnierimLab\\Ratterdam\\Figures\\R765{exp}\\"
datafile = f'E:\\Ratterdam\\R765\\R765{exp}\\'
behav = core.BehavioralData(datafile, exp, velocity_filter_thresh)
ts, position, alleyTracking, alleyVisits,  txtVisits = behav.loadData()
for subdir, dirs, fs in os.walk(datafile):
    for f in fs:
        if 'cl-maze' in f and 'OLD' not in f and 'Undefined' not in f:
            clustname = subdir[subdir.index("TT"):] + "\\" + f
            print(clustname)
            unit = core.UnitData(clustname, datafile, exp, alleyBounds, alleyVisits, txtVisits, position, ts)
            unit.loadData_raw()
            population[unit.name] = unit


TT10\cl-maze1.1


C:\Users\whockei1\Google Drive\KnierimLab\Ratterdam\Code\ratterdam_CoreDataStructures.py:139: RuntimeWarning: divide by zero encountered in reciprocal
  n = (hs*np.reciprocal(ho))*33
C:\Users\whockei1\Google Drive\KnierimLab\Ratterdam\Code\ratterdam_CoreDataStructures.py:139: RuntimeWarning: invalid value encountered in multiply
  n = (hs*np.reciprocal(ho))*33
C:\Users\whockei1\Google Drive\KnierimLab\Ratterdam\Code\ratterdam_CoreDataStructures.py:145: RuntimeWarning: divide by zero encountered in reciprocal
  n = (ls* np.reciprocal(lo)) * 33
C:\Users\whockei1\Google Drive\KnierimLab\Ratterdam\Code\ratterdam_CoreDataStructures.py:145: RuntimeWarning: invalid value encountered in multiply
  n = (ls* np.reciprocal(lo)) * 33
C:\Users\whockei1\Google Drive\Python_Code\KLab\mts_analysis\utility_fx.py:309: RuntimeWarning: invalid value encountered in true_divide
  Z=VV/WW


TT10\cl-maze1.2
TT10\cl-maze1.3
TT10\cl-maze1.4


C:\Users\whockei1\Google Drive\Python_Code\KLab\mts_analysis\utility_fx.py:305: RuntimeWarning: invalid value encountered in multiply
  W=0*U.copy()+1


TT10\cl-maze1.5
TT10\cl-maze1.6
TT10\cl-maze1.7
TT13\cl-maze1.1
TT13\cl-maze1.2
TT13\cl-maze1.3
TT15\cl-maze1.1
TT15\cl-maze1.2
TT15\cl-maze1.3
TT15\cl-maze1.4
TT15\cl-maze1.5
TT4\cl-maze1.1
TT4\cl-maze1.10
TT4\cl-maze1.11
TT4\cl-maze1.12
TT4\cl-maze1.2
TT4\cl-maze1.3
TT4\cl-maze1.4
TT4\cl-maze1.5
TT4\cl-maze1.6
TT4\cl-maze1.7
TT4\cl-maze1.8
TT4\cl-maze1.9
TT6\cl-maze1.1
TT6\cl-maze1.2
TT7\cl-maze1.1
TT7\cl-maze1.2
TT7\cl-maze1.3


In [14]:
unitList = list(population.keys())
unit = population[unitList[9]]
plt.scatter(unit.spikes[:,1], unit.spikes[:,2])

In [45]:
def calcOverallTestStat(linX, linY):
    """
    Given two test stats from all visits of a txt
    Take their binwise difference and return it
    """
    if type(linX) is np.ndarray and type(linY) is np.ndarray:
        d = linX - linY
        return d

In [46]:
banned = ['null']

In [47]:
allTestStatDiffs = np.empty((0))
allNullDiffs = np.empty((0))
minTestStatDiffs = np.empty((0))
for unit in population.values():
    for alley in [1,3,5,7,8,10,11,16,17]:
        pairs = ["AB", "BC", "CA"]
        for pair in pairs:
            x,y = unit.linRMS[alley][pair[0]], unit.linRMS[alley][pair[1]]
            diffTrace = calcOverallTestStat(x, y)
             
            labels = Perm.getLabels(unit, alley)
            null = Perm.genSingleNullStat(unit, alley, pair[0], pair[1], labels)
                
            if diffTrace is not None:
                allTestStatDiffs = np.concatenate((allTestStatDiffs, np.abs(diffTrace)))
                allNullDiffs     = np.concatenate((allNullDiffs, np.abs(null)))
                
            if unit.name not in banned and diffTrace is not None:
                minTestStatDiffs = np.concatenate((minTestStatDiffs, np.abs(diffTrace)))
                

allTestStatDiffs = allTestStatDiffs[np.isfinite(allTestStatDiffs)]
allNullDiffs = allNullDiffs[np.isfinite(allNullDiffs)]
minTestStatDiffs = minTestStatDiffs[np.isfinite(minTestStatDiffs)]

In [48]:
_ = plt.hist(allTestStatDiffs, bins=500,color='g',alpha=0.4)
_ = plt.hist(allNullDiffs, bins=500,color='k',alpha=0.4)
_ = plt.hist(minTestStatDiffs, bins=500,color='r',alpha=0.4)

for d,c in zip([allTestStatDiffs, allNullDiffs, minTestStatDiffs],['g','k','r']):
    plt.vlines(np.percentile(d,95), 0, 10000, f"{c}")

In [8]:
plt.xlabel("Test Statistic Size", fontsize=14)
plt.ylabel("Number of Bins", fontsize=14)
plt.title("Distribution of abs. test statistic values across population bins", fontsize=20)

Text(0.5,1,'Distribution of abs. test statistic values across population bins')

# Support Vector Machines - Beltway Data - Decoding different types of information

In [3]:
from sklearn.svm import SVC
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score, accuracy_score

C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [39]:
frThresh=0.1

In [47]:
# Alley and Texture - 27 label decoding

ratemaps = {alley: np.empty((0, Def.singleAlleyBins[0]-1)) for alley in [16,17,3,1,5,7,8,10,11]}
labels = []
popidx = []
idx = 0

target = 'Alley/Texture'
# accs, shuffs, oobs = [], [], []
for alley in [16,17,3,1,5,7,8,10,11]:
    for clustName, unit in population.items():
        popidx.append(idx)
        for stim in ['A', 'B', 'C']:
            rm = unit.linRMS[alley][stim]
            
            if type(rm) == np.ndarray and np.nanmax(rm)>frThresh:
                ratemaps[alley] = np.vstack((ratemaps[alley], rm))
                labels.append(f"{alley}{stim}")
                idx += 1
    for alley in [16,17,3,1,5,7,8,10,11]:
        ratemaps[alley][np.where(np.isnan(ratemaps[alley]))] = 0
        
pop = np.vstack(([ratemaps[i] for i in [16,17,3,1,5,7,8,10,11]]))
X = preprocessing.StandardScaler().fit_transform(pop)
Y = np.asarray(labels)
targetclasses = list(set(labels))

In [21]:
# alley alone
target = 'Alley'
targetclasses = [16,17,3,1,5,7,8,10,11]
ratemaps = {alley: np.empty((0, Def.singleAlleyBins[0]-1)) for alley in [1,3,5,7,8,10,11,16,17]}

for alley in [1,3,5,7,8,10,11,16,17]:
    for clustName, unit in population.items():
        for stim in ['A', 'B', 'C']:
            rm = unit.linRMS[alley][stim]
            if type(rm) == np.ndarray and np.nanmax(rm) > frThresh:
                ratemaps[alley] = np.vstack((ratemaps[alley], rm))
    for alley in [1,3,5,7,8,10,11,16,17]:
        ratemaps[alley][np.where(np.isnan(ratemaps[alley]))] = 0

pop = np.vstack(([ratemaps[i] for i in [1,3,5,7,8,10,11,16,17]]))
labels = [[i]*ratemaps[i].shape[0] for i in [1,3,5,7,8,10,11,16,17]]
labels = np.asarray([item for sublist in labels for item in sublist])
X = preprocessing.StandardScaler().fit_transform(pop)
Y = np.asarray(labels)

In [23]:
# texture alone
target = 'Texture'
targetclasses = ['A','B','C']
ratemaps = {txt: np.empty((0, Def.singleAlleyBins[0]-1)) for txt in ['A', 'B', 'C']}

for alley in [1,3,5,7,8,10,11,16,17]:
    for clustName, unit in population.items():
        for stim in ['A', 'B', 'C']:
            rm = unit.linRMS[alley][stim]
            if type(rm) == np.ndarray and np.nanmax(rm)>frThresh:
                ratemaps[stim] = np.vstack((ratemaps[stim], rm))
    for stim in ['A','B','C']:
        ratemaps[stim][np.where(np.isnan(ratemaps[stim]))] = 0

pop = np.vstack((ratemaps['A'], ratemaps['B'], ratemaps['C']))
labels = ['A']*ratemaps['A'].shape[0] + ['B']*ratemaps['B'].shape[0] + ['C']*ratemaps['C'].shape[0]
X = preprocessing.StandardScaler().fit_transform(pop)
Y = np.asarray(labels)

In [18]:
def compute_epoch(val,size):
    """hardcode that session is divided
    into thirds. find in which third the trial is in"""
    propthrusess = val/size
    if propthrusess < 0.5:
        epoch = 0
    elif propthrusess >= 0.5:
        epoch = 1
    return epoch

In [207]:
#alley/txt/epoch
target = "AlleyXTextureXEpoch"
ratemaps = np.empty((0, Def.singleAlleyBins[0]-1))
labels = []
for clustname, unit in population.items():
    for alley in [16,17,3,1,5,7,8,10,11]:
        for i,visit in enumerate(unit.alleys[alley]):
            rm = visit['ratemap1d']
            if type(rm) == np.ndarray and np.nanmax(rm) > frThresh:
                ratemaps = np.vstack((ratemaps, rm))
                epoch = compute_epoch(i,len(unit.alleys[alley]))
                labels.append(f"{alley}{visit['metadata']['stimulus']}{epoch}")

ratemaps = np.nan_to_num(ratemaps)
        
#X = preprocessing.StandardScaler().fit_transform(ratemaps)
X = preprocessing.Imputer().fit_transform(ratemaps)
Y = np.asarray(labels)
targetclasses = list(set(labels))

In [219]:
#alley/epoch
target = "AlleyXEpoch"
ratemaps = np.empty((0, Def.singleAlleyBins[0]-1))
labels = []
for clustname, unit in population.items():
    for alley in [16,17,3,1,5,7,8,10,11]:
        for i,visit in enumerate(unit.alleys[alley]):
            rm = visit['ratemap1d']
            if type(rm) == np.ndarray and np.nanmax(rm) > frThresh:
                ratemaps = np.vstack((ratemaps, rm))
                epoch = compute_epoch(i,len(unit.alleys[alley]))
                labels.append(f"{alley}{epoch}")

ratemaps = np.nan_to_num(ratemaps)
        
#X = preprocessing.StandardScaler().fit_transform(ratemaps)
X = preprocessing.Imputer().fit_transform(ratemaps)
Y = np.asarray(labels)
targetclasses = list(set(labels))

In [6]:
#txt/epoch
target = "TextureXEpoch"
ratemaps = np.empty((0, Def.singleAlleyBins[0]-1))
labels = []
for clustname, unit in population.items():
    for alley in [16,17,3,1,5,7,8,10,11]:
        for i,visit in enumerate(unit.alleys[alley]):
            rm = visit['ratemap1d']
            if type(rm) == np.ndarray and np.nanmax(rm) > frThresh:
                ratemaps = np.vstack((ratemaps, rm))
                epoch = compute_epoch(i,len(unit.alleys[alley]))
                labels.append(f"{visit['metadata']['stimulus']}{epoch}")

ratemaps = np.nan_to_num(ratemaps)
        
#X = preprocessing.StandardScaler().fit_transform(ratemaps)
X = preprocessing.Imputer().fit_transform(ratemaps)
Y = np.asarray(labels)
targetclasses = list(set(labels))

In [16]:
# splitting cells into train/test
precisions, recalls, f1s, accuracies = [], [], [], []
precisions_shuff, recalls_shuff, f1s_shuff, accuracies_shuff = [], [], [], []

avgType = 'macro'

target = 'AlleyXTexture'


for i in range(100):
    
    print(i)

    ratemaps_train = np.empty((0, Def.singleAlleyBins[0]-1))
    ratemaps_test = np.empty((0, Def.singleAlleyBins[0]-1))
    labels_train = []
    labels_test = []

    unitnames = list(population.keys())

    np.random.shuffle(unitnames) # KEY: HERE IS WHERE YOU SHUFFLE 

    poptrain = {name: population[name] for name in unitnames[:22]}
    poptest = {name: population[name] for name in unitnames[22:]}

    #create training set
    for clustname, unit in poptrain.items():
        for alley in [16,17,3,1,5,7,8,10,11]:
            for stim in ['A','B','C']:
                rm = unit.linRMS[alley][stim]
                if type(rm) == np.ndarray and np.nanmax(rm) > frThresh:
                    ratemaps_train = np.vstack((ratemaps_train, rm))
                    if target == 'Alley':
                        labels_train.append(f"{alley}")
                    elif target == 'Texture':
                        labels_train.append(f"{stim}")
                    elif target == 'AlleyXTexture':
                        labels_train.append(f"{alley}{stim}")
                        
    
    ratemaps_train[np.where(np.isnan(ratemaps_train))] = 0
    ratemaps_train = np.nan_to_num(ratemaps_train)
    Xtrain = preprocessing.StandardScaler().fit_transform(ratemaps_train)
    ytrain = np.asarray(labels_train)

    #create test set
    for clustname, unit in poptest.items():
        for alley in [16,17,3,1,5,7,8,10,11]:
            for stim in ['A','B','C']:
                rm = unit.linRMS[alley][stim]
                if type(rm) == np.ndarray and np.nanmax(rm) > frThresh:
                    ratemaps_test = np.vstack((ratemaps_test, rm))
                    if target == 'Alley':
                        labels_test.append(f"{alley}")
                    elif target == 'Texture':
                        labels_test.append(f"{stim}")
                    elif target == 'AlleyXTexture':
                        labels_test.append(f"{alley}{stim}")
                        

    ratemaps_test[np.where(np.isnan(ratemaps_test))] = 0
    ratemaps_test = np.nan_to_num(ratemaps_test)
    Xtest = preprocessing.StandardScaler().fit_transform(ratemaps_test)
    ytest = np.asarray(labels_test)

    targetclasses = list(set(labels_train) | set(labels_test))

    _ = np.random.shuffle(labels_train)
    
    svc = SVC(C=1e7, gamma=0.01,kernel='rbf')
    svc.fit(Xtrain,ytrain)
    yfit = svc.predict(Xtest)
    
    svc_shuffle = SVC(C=1e7, gamma=0.01, kernel='rbf')
    svc_shuffle.fit(Xtrain,labels_train) # it has been shuffled in place
    yfit_shuff = svc_shuffle.predict(Xtest)

    
    
    p = precision_score(ytest, yfit, average=avgType)
    r = recall_score(ytest, yfit, average=avgType)
    f1 = f1_score(ytest, yfit, average=avgType)
    acc = accuracy_score(ytest,yfit)
    precisions.append(p)
    recalls.append(r)
    f1s.append(f1)
    accuracies.append(acc)
    
    pshuff = precision_score(ytest, yfit_shuff, average=avgType)
    rshuff = recall_score(ytest, yfit_shuff, average=avgType)
    f1shuff = f1_score(ytest, yfit_shuff, average=avgType)
    accshuff = accuracy_score(ytest,yfit_shuff)
    precisions_shuff.append(pshuff)
    recalls_shuff.append(rshuff)
    f1s_shuff.append(f1shuff)
    accuracies_shuff.append(accshuff)

0


C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31


C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [49]:
def condense_epochs(ratemaps, labels):
    """ratemaps is dict w keys being alleys and values
    being a stack of linear ratemaps. labels is array stack
    with target label for each ratemap if the dict values
    were stacked in one big array in order of beltway alleys
    ie. [16,17,1,5,7,8,10,11]"""
    arrstack = np.vstack(([ratemaps[i] for i in [16,17,3,1,5,7,8,10,11]]))
    condensed_ratemaps = {alley: np.empty((0, Def.singleAlleyBins[0]-1)) for alley in [16,17,3,1,5,7,8,10,11]}
    arrlb = [(arr, lab) for arr,lab in zip(arrstack,labels)]
    arrlb = sorted(arrlb, key=lambda x: x[1])
    for groupID, rows in groupby(arrlb, key = itemgetter(1)):
        condensed_ratemaps[group] = np.ma.masked_invalid(rows).mean(axis=0)
    
    return condensed_ratemaps

In [20]:
# splitting cells into train/test FOR EPOCH STUFF WHICH IS TRIAL BASED 
precisions, recalls, f1s, accuracies = [], [], [], []
precisions_shuff, recalls_shuff, f1s_shuff, accuracies_shuff = [], [], [], []

avgType = 'macro'

target = 'AlleyXEpoch'


for i in range(50):
    
    print(i)

    ratemaps_train = {alley: np.empty((0, Def.singleAlleyBins[0]-1)) for alley in [16,17,3,1,5,7,8,10,11]}
    ratemaps_test = {alley: np.empty((0, Def.singleAlleyBins[0]-1)) for alley in [16,17,3,1,5,7,8,10,11]}
    labels_train = []
    labels_test = []

    unitnames = list(population.keys())

    np.random.shuffle(unitnames) # KEY: HERE IS WHERE YOU SHUFFLE 

    poptrain = {name: population[name] for name in unitnames[:22]}
    poptest = {name: population[name] for name in unitnames[22:]}

    #train
    for clustname, unit in poptrain.items():
        for alley in [16,17,3,1,5,7,8,10,11]:
            ratemaps_temp = {alley: np.empty((0, Def.singleAlleyBins[0]-1)) for alley in [16,17,3,1,5,7,8,10,11]}
            labels_temp = []
            for i,visit in enumerate(unit.alleys[alley]):
                rm = visit['ratemap1d']
                if type(rm) == np.ndarray and np.nanmax(rm) > frThresh:
                    ratemaps_temp[alley] = np.vstack((ratemaps_temp[alley], rm))
                    epoch = compute_epoch(i,len(unit.alleys[alley]))
                    if target == 'AlleyXEpoch':
                        labels_temp.append(f"{alley}{epoch}")
                    elif target == 'TextureXEpoch':
                        labels_temp.append(f"{visit['metadata']['stimulus']}{epoch}")
                    elif target == 'AlleyXTextureXEpoch':
                        labels_temp.append(f"{alley}{visit['metadata']['stimulus']}{epoch}")
    
            ratemaps_train[alley] = np.vstack((ratemaps_train[alley],npcondense_epochs(ratemaps_temp, labels_temp)))
            labels_train.extend(labels_temp)
                        
        for alley in [16,17,3,1,5,7,8,10,11]:
            ratemaps_train[alley][np.where(np.isnan(ratemaps_train[alley]))] = 0

    poptrain = np.vstack(([ratemaps_train[i] for i in [16,17,3,1,5,7,8,10,11]]))
    Xtrain = preprocessing.Imputer().fit_transform(poptrain)
    ytrain = np.asarray(labels_train)

    #test
    for clustname, unit in poptest.items():
        for alley in [16,17,3,1,5,7,8,10,11]:
            for i,visit in enumerate(unit.alleys[alley]):
                rm = visit['ratemap1d']
                if type(rm) == np.ndarray and np.nanmax(rm) > frThresh:
                    ratemaps_test[alley] = np.vstack((ratemaps_test[alley], rm))
                    epoch = compute_epoch(i,len(unit.alleys[alley]))
                    if target == 'AlleyXEpoch':
                        labels_test.append(f"{alley}{epoch}")
                    elif target == 'TextureXEpoch':
                        labels_test.append(f"{visit['metadata']['stimulus']}{epoch}")
                    elif target == 'AlleyXTextureXEpoch':
                        labels_test.append(f"{alley}{visit['metadata']['stimulus']}{epoch}")
                        
        for alley in [16,17,3,1,5,7,8,10,11]:
            ratemaps_test[alley][np.where(np.isnan(ratemaps_test[alley]))] = 0

    poptest = np.vstack(([ratemaps_test[i] for i in [16,17,3,1,5,7,8,10,11]]))
    Xtest = preprocessing.Imputer().fit_transform(poptest)
    ytest = np.asarray(labels_test)

    targetclasses = list(set(labels_train) | set(labels_test))

    _ = np.random.shuffle(labels_train)
    
    svc = SVC(C=1e7, gamma=0.01,kernel='rbf')
    svc.fit(Xtrain,ytrain)
    yfit = svc.predict(Xtest)
    
    svc_shuffle = SVC(C=1e7, gamma=0.01, kernel='rbf')
    svc_shuffle.fit(Xtrain,labels_train) # it has been shuffled in place
    yfit_shuff = svc_shuffle.predict(Xtest)

    
    
    p = precision_score(ytest, yfit, average=avgType)
    r = recall_score(ytest, yfit, average=avgType)
    f1 = f1_score(ytest, yfit, average=avgType)
    acc = accuracy_score(ytest,yfit)
    precisions.append(p)
    recalls.append(r)
    f1s.append(f1)
    accuracies.append(acc)
    
    pshuff = precision_score(ytest, yfit_shuff, average=avgType)
    rshuff = recall_score(ytest, yfit_shuff, average=avgType)
    f1shuff = f1_score(ytest, yfit_shuff, average=avgType)
    accshuff = accuracy_score(ytest,yfit_shuff)
    precisions_shuff.append(pshuff)
    recalls_shuff.append(rshuff)
    f1s_shuff.append(f1shuff)
    accuracies_shuff.append(accshuff)

0


ValueError: X.shape[1] = 6 should be equal to 7, the number of features at training time

In [17]:
fig, ax = plt.subplots(2,2)

ax[0,0].hist(accuracies, color='b', alpha=0.7)
ax[0,0].hist(accuracies_shuff, color='k', alpha=0.7)
ax[0,0].vlines(np.mean(accuracies), ax[0,0].get_ylim()[0], ax[0,0].get_ylim()[1],color='b')
ax[0,0].vlines(np.percentile(accuracies_shuff,95), ax[0,0].get_ylim()[0], ax[0,0].get_ylim()[1])
ax[0,0].set_title("Accuracy")

ax[0,1].hist(precisions, color='b', alpha=0.7)
ax[0,1].hist(precisions_shuff, color='k', alpha=0.7)
ax[0,1].vlines(np.mean(precisions), ax[0,0].get_ylim()[0], ax[0,0].get_ylim()[1],color='b')
ax[0,1].vlines(np.percentile(precisions_shuff,95), ax[0,0].get_ylim()[0], ax[0,0].get_ylim()[1])
ax[0,1].set_title("Precision")

ax[1,0].hist(recalls, color='b', alpha=0.7)
ax[1,0].hist(recalls_shuff, color='k', alpha=0.7)
ax[1,0].vlines(np.mean(recalls), ax[0,0].get_ylim()[0], ax[0,0].get_ylim()[1],color='b')
ax[1,0].vlines(np.percentile(recalls_shuff,95), ax[0,0].get_ylim()[0], ax[0,0].get_ylim()[1])
ax[1,0].set_title("Recall")

ax[1,1].hist(f1s, color='b', alpha=0.7)
ax[1,1].hist(f1s_shuff, color='k', alpha=0.7)
ax[1,1].vlines(np.mean(f1s), ax[0,0].get_ylim()[0], ax[0,0].get_ylim()[1],color='b')
ax[1,1].vlines(np.percentile(f1s_shuff,95), ax[0,0].get_ylim()[0], ax[0,0].get_ylim()[1])
ax[1,1].set_title("F1 Score")

plt.suptitle(target)


Text(0.5,0.98,'AlleyXTexture')

In [ ]:
# Find optimal C/gamma values

Xtrain, Xtest, ytrain, ytest = train_test_split(X, Y, shuffle=True, random_state=0)
tuned_parameters = [{'kernel': ['rbf'], 
                     'gamma': [1e-3, 1e-4, 1e-2, 1e-1, 1e-0],
                     'C': [1e3, 1e4, 1e5, 1e6, 1e7, 1e8, 1e9]},]

clf = GSCV(SVC(), tuned_parameters, cv=5)
clf.fit(Xtrain,ytrain)

In [ ]:
print("Best parameters set found on development set:")
print()
print(clf.best_params_)
print()
print("Grid scores on development set:")
print()
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
          % (mean, std * 2, params))
print()

print("Detailed classification report:")
print()
print("The model is trained on the full development set.")
print("The scores are computed on the full evaluation set.")
print()
y_true, y_pred = ytest, clf.predict(Xtest)
print(classification_report(y_true, y_pred))
print()

### ROC Analysis

In [179]:
# ROC Curves

y=Y

y = label_binarize(y, classes=targetclasses)
n_classes = y.shape[1]

# shuffle and split training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.5,
                                                    random_state=0)

# Learn to predict each class against the other
classifier = OneVsRestClassifier(svm.SVC(kernel='rbf',C=1e7,gamma=0.01, probability=True,
                                 random_state=0))
y_score = classifier.fit(X_train, y_train).decision_function(X_test)

# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_score.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

In [180]:
# Compute macro-average ROC curve and ROC area

# First aggregate all false positive rates
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

# Then interpolate all ROC curves at this points
mean_tpr = np.zeros_like(all_fpr)
for i in range(n_classes):
    mean_tpr += interp(all_fpr, fpr[i], tpr[i])

# Finally average it and compute AUC
mean_tpr /= n_classes

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])


plt.figure()
lw=2
plt.plot(fpr["micro"], tpr["micro"],
         label='micro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["micro"]),
         color='deeppink', linestyle=':', linewidth=4)

plt.plot(fpr["macro"], tpr["macro"],
         label='macro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["macro"]),
         color='navy', linestyle=':', linewidth=4)

colors = np.random.choice(list(plt.cm.colors.CSS4_COLORS.keys()),n_classes,replace=False)
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=lw,
             label='ROC curve of class {0} (area = {1:0.2f})'
             ''.format(i, roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--', lw=lw)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title(f'{target} Some extension of Receiver operating characteristic to multi-class')
plt.legend(loc="lower right")

In [65]:
# Alley and Texture - 27 label decoding

ratemaps = {alley: np.empty((0, Def.singleAlleyBins[0]-1)) for alley in [16,17,3,1,5,7,8,10,11]}
labels = []
popidx = []
idx = 0

target = 'Alley/Texture'
# accs, shuffs, oobs = [], [], []
for alley in [16,17,3,1,5,7,8,10,11]:
    for clustName, unit in population.items():
        popidx.append(idx)
        for stim in ['A', 'B', 'C']:
            rm = unit.linRMS[alley][stim]
            
            if type(rm) == np.ndarray and np.nanmax(rm)>frThresh:
                ratemaps[alley] = np.vstack((ratemaps[alley], rm))
                #labels.append(f"{alley}{np.random.choice(['A','B','C'])}")
                labels.append(f"{alley}{np.random.choice(['A','B','C'])}")
                idx += 1
    for alley in [16,17,3,1,5,7,8,10,11]:
        ratemaps[alley][np.where(np.isnan(ratemaps[alley]))] = 0
        
pop = np.vstack(([ratemaps[i] for i in [16,17,3,1,5,7,8,10,11]]))
X = preprocessing.StandardScaler().fit_transform(pop)
Y = np.asarray(labels)
targetclasses = list(set(labels))

### Metrics: Confusion Matrices and Precision/Recall/F1/Accuracy

In [66]:
# this is for when youre creating test/train by randomizing obsv

precisions, recalls, f1s, accuracies = [], [], [], []
avgType = 'macro'
for i in range(50):
    print(i)
    
    Xtrain, Xtest, ytrain, ytest = train_test_split(X, Y, shuffle=True) #default split size is 1/4
    svc = SVC(C=1e7, gamma=0.01,kernel='rbf')
    svc.fit(Xtrain,ytrain)
    yfit = svc.predict(Xtest)
    p = precision_score(ytest, yfit, average=avgType)
    r = recall_score(ytest, yfit, average=avgType)
    f1 = f1_score(ytest, yfit, average=avgType)
    acc = accuracy_score(ytest,yfit)
    precisions.append(p)
    recalls.append(r)
    f1s.append(f1)
    accuracies.append(acc)
print(f"Average precision: {np.mean(precisions)}")
print(f"Average recalls: {np.mean(recalls)}")
print(f"Average F1 score: {np.mean(f1s)}")
print(f"Average accuracy: {np.mean(accuracies)}")

0
1
2


C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


3
4
5
6
7


C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
Average precision: 0.08930699270656536
Average recalls: 0.0956384500273389
Average F1 score: 0.08249653720490693
Average accuracy: 0.09074380165289257


In [67]:
fig, ax = plt.subplots(2,2)

ax[0,0].hist(accuracies, color='b', alpha=0.7)
#ax[0,0].hist(accuracies_shuff, color='k', alpha=0.7)
ax[0,0].vlines(np.mean(accuracies), ax[0,0].get_ylim()[0], ax[0,0].get_ylim()[1],color='b')
#ax[0,0].vlines(np.percentile(accuracies_shuff,95), ax[0,0].get_ylim()[0], ax[0,0].get_ylim()[1])
ax[0,0].set_title("Accuracy")

ax[0,1].hist(precisions, color='b', alpha=0.7)
#ax[0,1].hist(precisions_shuff, color='k', alpha=0.7)
ax[0,1].vlines(np.mean(precisions), ax[0,0].get_ylim()[0], ax[0,0].get_ylim()[1],color='b')
#ax[0,1].vlines(np.percentile(precisions_shuff,95), ax[0,0].get_ylim()[0], ax[0,0].get_ylim()[1])
ax[0,1].set_title("Precision")

ax[1,0].hist(recalls, color='b', alpha=0.7)
#ax[1,0].hist(recalls_shuff, color='k', alpha=0.7)
ax[1,0].vlines(np.mean(recalls), ax[0,0].get_ylim()[0], ax[0,0].get_ylim()[1],color='b')
#ax[1,0].vlines(np.percentile(recalls_shuff,95), ax[0,0].get_ylim()[0], ax[0,0].get_ylim()[1])
ax[1,0].set_title("Recall")

ax[1,1].hist(f1s, color='b', alpha=0.7)
#ax[1,1].hist(f1s_shuff, color='k', alpha=0.7)
ax[1,1].vlines(np.mean(f1s), ax[0,0].get_ylim()[0], ax[0,0].get_ylim()[1],color='b')
#ax[1,1].vlines(np.percentile(f1s_shuff,95), ax[0,0].get_ylim()[0], ax[0,0].get_ylim()[1])
ax[1,1].set_title("F1 Score")

plt.suptitle(target)


Text(0.5,0.98,'Alley/Texture')

In [59]:
plt.figure()

In [229]:
accs = accuracies
fig, ax = plt.subplots(1,2)
ax[0].hist(accs)
ax[0].set_xlabel("1/4 Cross-Val Accuracy")
ax[0].set_ylabel('Frequency')
ax[1].plot(accs)
ax[1].plot([0, nruns], [np.mean(accs), np.mean(accs)], 'k--',alpha=0.5)
ax[1].set_xlabel("Iteration of Repeated Cross-Validation")
ax[1].set_ylabel("Accuracy")
plt.suptitle(f"SVM Performance, {target}, C=1e7, gamma=0.01, RBF kernel", fontsize=16)
ax[1].hlines(1/len(targetclasses),0,nruns,'k')
ax[0].vlines(1/len(targetclasses),0,ax[0].get_ylim()[1],'k')

In [28]:
def plot_confusion_matrix(X, Y, numRuns, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    
    Taken from the sklearn docs. Modified by WH
    """
    if not title:
        if normalize:
            title = f'Normalized confusion matrix, {numRuns} runs'
        else:
            title = f'Confusion matrix, without normalization, {numRuns} runs'

    # Compute multiple confusion matrces and sum
    allcms = []
    for run in range(numRuns):
        Xtrain, Xtest, ytrain, y_true = train_test_split(X, Y, shuffle=True, train_size=0.75)
        svc = SVC(C=1e7, gamma=0.01,kernel='rbf')
        svc.fit(Xtrain,ytrain)
        y_pred = svc.predict(Xtest)
        cm = confusion_matrix(y_true, y_pred)
        allcms.append(cm)
    
    cm = np.sum(np.asarray(allcms), axis=0)
        
    # Only use the labels that appear in the data
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')
    
    ax.plot(range(classes.shape[0]), range(classes.shape[0]))

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax

In [46]:
plot_confusion_matrix(X,Y,100, classes=np.unique(Y), normalize=False,
                      title=f'{target} Confusion matrix, No R, {frThresh}Hz Fr Thresh')

C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


ValueError: operands could not be broadcast together with shapes (27,27) (26,26) 

In [29]:
%qtconsole --style native